# Libraries

In [1]:
library(tidyverse)
library(forecast)
library(ggplot2)
library(dplyr)
library(data.table)
library(IRdisplay)

library(foreach)
library(doParallel)

library(caret)
library(randomForest)
library(xgboost)
library(progress)


Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Registered S3 methods overwritten by 'forecast':
  method             from    
  fitted.fracdiff    fracdiff
  residuals.fracdiff fracdiff

Attaching package: 'data.table'

The following ob

# Custom functions

### Display tables

In [2]:
# Custom display function for the first and last 5 rows or full table if <= 20 rows
display_limited <- function(dt) {
  n <- nrow(dt)
  
  # If there are 20 or fewer rows, display the full table
  if (n <= 20) {
    limited_dt <- dt
  } else {
    # Otherwise, concatenate the first 5 rows, '...' and the last 5 rows
    limited_dt <- rbind(head(dt, 5), as.list(rep("...", ncol(dt))), tail(dt, 5))
  }
  
  # Generate raw HTML manually
  html_output <- paste0(
    "<table border='1' style='border-collapse:collapse;'>",
    "<thead><tr>",
    paste0("<th>", colnames(limited_dt), "</th>", collapse = ""),
    "</tr></thead>",
    "<tbody>",
    paste0(
      apply(limited_dt, 1, function(row) {
        paste0("<tr>", paste0("<td>", row, "</td>", collapse = ""), "</tr>")
      }),
      collapse = ""
    ),
    "</tbody></table>"
  )
  
  # Display the HTML in the Jupyter notebook
  display_html(html_output)
}


### Calculate metrics

In [3]:
calculate_metrics <- function(R_t, R_hat_t, individual) {
  # Ensure the inputs are numeric vectors and individual is a dataframe
  if (!is.numeric(R_t) || !is.numeric(R_hat_t)) {
    stop("Both R_t and R_hat_t need to be numeric vectors.")
  }
  
  # Calculate metrics
  mae <- mean(abs(R_t - R_hat_t), na.rm = TRUE)
  rmse <- sqrt(mean((R_t - R_hat_t)^2, na.rm = TRUE))
  mape <- mean(abs((R_t - R_hat_t) / R_t), na.rm = TRUE) * 100
  r_squared <- ifelse(all(R_t == R_hat_t), 1, summary(lm(R_t ~ R_hat_t))$r.squared)
  
  # Create a data frame to hold the metrics and values
  metrics_table <- data.frame(
    MAE = mae,
    RMSE = rmse,
    MAPE = mape,
    R_squared = r_squared
  )
  
  # Return the metrics table
  return(metrics_table)
}

### Data Preparation

In [4]:
prepare_X_t <- function(individual) {
  # Ensure the input is a dataframe
  if (!is.data.frame(individual)) {
    stop("The input must be a dataframe.")
  }
  
  # Extract hour from start_time and create a 'time_of_day' column
  individual$time_of_day <- format(as.POSIXct(individual$HourDK), "%H:%M:%S")
  
  # Exclude specified columns but keep 'time_of_day'
  X_t <- subset(individual, select = -c(HourDK, GrossConsumptionMWh))
  
  # Convert month, weekday, and time_of_day to factors with a reference category
  X_t$month <- relevel(as.factor(X_t$MonthOfYear), ref = "December")  # Set December as reference
  X_t$weekday <- relevel(as.factor(X_t$DayOfWeek), ref = "Sunday")   # Set Sunday as reference 
  X_t$time_of_day <- relevel(as.factor(X_t$Hour), ref = "0")         # Set 23 (11 PM) as reference

  # Remove original 'MonthOfYear', 'DayOfWeek', and 'Hour' columns to avoid duplication
  X_t <- subset(X_t, select = -c(MonthOfYear, DayOfWeek, Hour))
  
  # Create dummy variables for all factor columns (excluding reference levels)
  X_t <- model.matrix(~ . - 1, data = X_t)
  
  # Find the column indices for numerical columns AFTER creating dummy variables
  num_cols <- grep("^(Electric cars|Plug-in hybrid cars|humidity_past1h|temp_mean_past1h|wind_speed_past1h|EL_price)", colnames(X_t))
  
  # Standardize selected numerical columns
  X_t[, num_cols] <- apply(X_t[, num_cols], 2, 
                           function(x) (x - min(x)) / (max(x) - min(x)))
  
  # Return the processed dataframe
  return(as.data.frame(X_t))
}


### Lag and Align data by \\(h\\) (horizon)

In [5]:
lag_and_align_data <- function(X_t, R_t, h = 1) {
  # Validate inputs
  if (!is.numeric(R_t)) {
    stop("R_t should be a numeric vector.")
  }
  if (!is.data.frame(X_t) && !is.matrix(X_t)) {
    stop("X_t should be a dataframe or a matrix.")
  }
  if (!is.numeric(h) || h < 1) {
    stop("h should be a positive integer.")
  }
  
  # Convert X_t to a dataframe if it's a matrix
  if (is.matrix(X_t)) {
    X_t <- as.data.frame(X_t)
  }
  
  # Align R_t with the lagged X_t
  # Shift R_t by h positions to align with X_t from the previous timestep
  R_t_aligned <- R_t[(h + 1):length(R_t)]
  
  # Keep X_t up to the second to last row, so it aligns with the shifted R_t
  X_t_aligned <- X_t[1:(nrow(X_t) - h), ]
  
  # Return the aligned datasets
  list(X_t = X_t_aligned, R_t = R_t_aligned)
}

### Plot actual vs estimated

In [6]:
plot_actual_vs_estimated <- function(R_t, R_hat_t, individual) {
  # Validate input
  if (!is.numeric(R_t) || !is.numeric(R_hat_t)) {
    stop("R_t and R_hat_t should be numeric vectors.")
  }
  if (!is.data.frame(individual)) {
    stop("individual should be a dataframe.")
  }
    
  # Create the plot
  plot(R_t, type = 'l', col = 'blue', xlab = "Time", ylab = "Value", 
       main = "Actual vs. Estimated Time Series\nelvarme: %s, zip_code: %s")
  lines(R_hat_t, type = 'l', col = 'red')
  legend("topleft", legend = c("Actual", "Estimated"), col = c("blue", "red"), lty = 1)
}

# Loading data

In [7]:
##### Setting workign directory and loadign data #####
base_path <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data Cleaning"
setwd(base_path)
data <- fread(paste0(base_path,"/Output_file.csv"))
MSTL <- fread(paste0(base_path,"/MSTL_decomp_results.csv"))

# Loop for rounds

In [9]:


train_size    <- 17544 # 2-year training set
# train_size  <- 8784  # 1-year training set
num_timesteps <- 720
h             <- 1
total_size    <- nrow(data) - h
set.seed(42)

tune_grid <- expand.grid(
  eta = c(0.01, 0.05, 0.1),
  max_depth = c(3, 6, 9),
  subsample = c(0.6, 0.8, 1.0),
  colsample_bytree = c(0.6, 0.8, 1.0)
)

# Set up directory paths
#path_R <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Rounds/R_hat"
#path_M <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Rounds/Metrics"
path_R <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Round no window/R_hat"
path_M <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Round no window/Metrics"

# Number of cores for parallel processing
no_cores <- detectCores() - 1
cl <- makeCluster(no_cores)
registerDoParallel(cl)

train_size    <- 17544 #2 year training set
#train_size    <- 8784  #1 year training set
h             <- 1
total_size    <- nrow(data)-h
num_timesteps <- total_size - train_size


nrounds       <- 100
set.seed(42) 

individual <- data
X_t <- prepare_X_t(as.data.frame(individual))
R_t <- as.matrix(MSTL$Remainder, nrow = nrow(MSTL), ncol = 1)

lag_and_align <- lag_and_align_data(X_t, R_t)
X_t <- as.matrix(lag_and_align$X_t)
R_t <- as.numeric(lag_and_align$R_t)

# Simple train-validation split for hyperparameter tuning
train_index <- 1:train_size
val_index <- (train_size + 1):(train_size + num_timesteps)

dtrain <- xgb.DMatrix(data = X_t[train_index, ], label = R_t[train_index])
dval <- xgb.DMatrix(data = X_t[val_index, ], label = R_t[val_index])

watchlist <- list(train = dtrain, eval = dval)

tune_grid <- expand.grid(
  eta = c(0.01, 0.05, 0.1),
  max_depth = c(3, 6, 9),
  subsample = c(0.6, 0.8, 1.0),
  colsample_bytree = c(0.6, 0.8, 1.0)
)

# Initialize progress bar
num_models <- length(seq(50, 5000, by = 50))
pb <- progress_bar$new(
  total = num_models,
  format = "Tuning model [:bar] :percent in :elapsed"
)

# Outer loop to iterate through nrounds from 50 to 10,000 in increments of 50
for (nrounds in seq(50, 5000, by = 50)) {
  pb$tick()
  
  best_rmse <- Inf
  best_params <- NULL
  
  # Grid search for hyperparameter tuning
  for (i in 1:nrow(tune_grid)) {
    params <- list(
      objective = "reg:squarederror",
      eta = tune_grid$eta[i],
      max_depth = tune_grid$max_depth[i],
      subsample = tune_grid$subsample[i],
      colsample_bytree = tune_grid$colsample_bytree[i]
    )
    
    xgb_model <- xgb.train(
      params = params,
      data = dtrain,
      nrounds = nrounds,
      watchlist = watchlist,
      early_stopping_rounds = 5,
      verbose = 0
    )
    
    if (xgb_model$best_score < best_rmse) {
      best_rmse <- xgb_model$best_score
      best_params <- params
    }
  }

  cl <- makeCluster(no_cores)
  registerDoParallel(cl)
    
  # Parallel processing for fitting and generating results
  results <- foreach(j = seq(1, nrow(X_t) - train_size, by = num_timesteps), .combine = 'c', .packages = 'xgboost') %dopar% {
    start_index <- j
    end_index <- j + train_size - 1
    
    train_X_t <- X_t[start_index:end_index, ]
    train_R_t <- R_t[start_index:end_index]
    
    dtrain <- xgb.DMatrix(data = train_X_t, label = train_R_t)
    
    xgb_model <- xgb.train(params = best_params, data = dtrain, nrounds = nrounds)
    
    test_start_index <- end_index + 1
    test_end_index <- min(end_index + num_timesteps, total_size)
    test_X_t <- X_t[test_start_index:test_end_index, , drop = FALSE]
    dtest <- xgb.DMatrix(data = test_X_t)
    
    test_predictions <- predict(xgb_model, newdata = dtest)
    
    num_predictions_to_return <- min(num_timesteps, total_size - test_start_index + 1)
    return(test_predictions[1:num_predictions_to_return])
  }
  stopCluster(cl)
  
  R_hat_t <- unlist(results)
  individual_metrics <- calculate_metrics(tail(R_t, n = length(R_t) - train_size), R_hat_t, data)
  
  # Save metrics and R_hat_t to CSV files
  file_name_R_hat_t <- file.path(path_R, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_XGB_R_hat_t.csv"))
  file_name_metrics <- file.path(path_M, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_XGB_Metrics.csv"))
  
  write.csv(R_hat_t, file = file_name_R_hat_t, row.names = FALSE)
  write.csv(individual_metrics, file = file_name_metrics, row.names = FALSE)
}

In [ ]:
2+2

In [ ]:
2+2

In [10]:


train_size    <- 17544 # 2-year training set
# train_size  <- 8784  # 1-year training set
num_timesteps <- 720
h             <- 1
total_size    <- nrow(data) - h
set.seed(42)

tune_grid <- expand.grid(
  eta = c(0.01, 0.05, 0.1),
  max_depth = c(3, 6, 9),
  subsample = c(0.6, 0.8, 1.0),
  colsample_bytree = c(0.6, 0.8, 1.0)
)

# Set up directory paths
#path_R <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Rounds/R_hat"
#path_M <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Rounds/Metrics"
path_R <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Round no window/R_hat"
path_M <- "C:/Users/madsh/OneDrive/Dokumenter/kandidat/Fællesmappe/Forecasting-energy-consumption/Data/Results/Boosting/Round no window/Metrics"

# Number of cores for parallel processing
no_cores <- detectCores() - 1
cl <- makeCluster(no_cores)
registerDoParallel(cl)

train_size    <- 17544 #2 year training set
#train_size    <- 8784  #1 year training set
h             <- 1
total_size    <- nrow(data)-h
num_timesteps <- total_size - train_size


nrounds       <- 100
set.seed(42) 

individual <- data
X_t <- prepare_X_t(as.data.frame(individual))
R_t <- as.matrix(MSTL$Remainder, nrow = nrow(MSTL), ncol = 1)

lag_and_align <- lag_and_align_data(X_t, R_t)
X_t <- as.matrix(lag_and_align$X_t)
R_t <- as.numeric(lag_and_align$R_t)

# Simple train-validation split for hyperparameter tuning
train_index <- 1:train_size
val_index <- (train_size + 1):(train_size + num_timesteps)

dtrain <- xgb.DMatrix(data = X_t[train_index, ], label = R_t[train_index])
dval <- xgb.DMatrix(data = X_t[val_index, ], label = R_t[val_index])

watchlist <- list(train = dtrain, eval = dval)

tune_grid <- expand.grid(
  eta = c(0.01, 0.05, 0.1),
  max_depth = c(3, 6, 9),
  subsample = c(0.6, 0.8, 1.0),
  colsample_bytree = c(0.6, 0.8, 1.0)
)

# Initialize progress bar
num_models <- length(seq(50, 5000, by = 50))
pb <- progress_bar$new(
  total = num_models,
  format = "Tuning model [:bar] :percent in :elapsed"
)

# Outer loop to iterate through nrounds from 50 to 10,000 in increments of 50
for (nrounds in seq(50, 5000, by = 50)) {
  pb$tick()
  
  best_rmse <- Inf
  best_params <- NULL
  
  # Grid search for hyperparameter tuning
  for (i in 1:nrow(tune_grid)) {
    params <- list(
      objective = "reg:squarederror",
      eta = tune_grid$eta[i],
      max_depth = tune_grid$max_depth[i],
      subsample = tune_grid$subsample[i],
      colsample_bytree = tune_grid$colsample_bytree[i]
    )
    
    xgb_model <- xgb.train(
      params = params,
      data = dtrain,
      nrounds = nrounds,
      watchlist = watchlist,
      early_stopping_rounds = 5,
      verbose = 0
    )
    
    if (xgb_model$best_score < best_rmse) {
      best_rmse <- xgb_model$best_score
      best_params <- params
    }
  }

# Number of cores for parallel processing within xgboost
no_cores <- detectCores() - 1

# Prepare training data
dtrain <- xgb.DMatrix(data = X_t[1:train_size, ], label = R_t[1:train_size])

# Configure best parameters and include parallel processing
best_params$nthread <- no_cores  # Use all available cores

# Train the xgboost model
xgb_model <- xgb.train(
  params = best_params,
  data = dtrain,
  nrounds = nrounds,
  watchlist = watchlist,
  early_stopping_rounds = 5,
  verbose = 0
)

# Prepare test data and make predictions
test_start_index <- train_size + 1
test_end_index <- min(train_size + num_timesteps, total_size)
test_X_t <- X_t[test_start_index:test_end_index, , drop = FALSE]
dtest <- xgb.DMatrix(data = test_X_t)

# Predict using the trained model
test_predictions <- predict(xgb_model, newdata = dtest)

# Return predictions for the length needed
num_predictions_to_return <- min(num_timesteps, total_size - train_size)
R_hat_t <- test_predictions[1:num_predictions_to_return]

# Calculate individual metrics
individual_metrics <- calculate_metrics(tail(R_t, n = length(R_t) - train_size), R_hat_t, data)

# Save metrics and R_hat_t to CSV files
file_name_R_hat_t <- file.path(path_R, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_XGB_R_hat_t.csv"))
file_name_metrics <- file.path(path_M, paste0("h=", h, "_steps_ahead=", num_timesteps, "_nrounds=", nrounds, "_train_size=", train_size, "_XGB_Metrics.csv"))

write.csv(R_hat_t, file = file_name_R_hat_t, row.names = FALSE)
write.csv(individual_metrics, file = file_name_metrics, row.names = FALSE)
}